# Basic Examples for Model with Metadata


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-seldon" modified.


## 1. Directly define in model

In [3]:
%%writefile models/init-metadata/Model.py

import logging
import json
import os


class Model:
    def predict(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return features

    def init_metadata(self):
        logging.info("metadata method  called")

        meta = {
            "name": "my-model-name",
            "versions": ["my-model-version-01"],
            "platform": "seldon",
            "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 5]}],
            "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}],
        }

        return meta

Overwriting models/init-metadata/Model.py


In [4]:
%%writefile model-metadata/init-metadata.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metadata:0.3
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
    graph:
      children: []
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/init-metadata.yaml


In [5]:
%%bash

kubectl apply -f model-metadata/init-metadata.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model created
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [7]:
!curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model | jq .

{
  "inputs": [
    {
      "datatype": "BYTES",
      "name": "input",
      "shape": [
        1,
        5
      ]
    }
  ],
  "name": "my-model-name",
  "outputs": [
    {
      "datatype": "BYTES",
      "name": "output",
      "shape": [
        1
      ]
    }
  ],
  "platform": "seldon",
  "versions": [
    "my-model-version-01"
  ]
}


In [8]:
import json

meta = !curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model 
meta = json.loads(meta[0])

assert meta == {
    "name": "my-model-name",
    "platform": "seldon",
    "versions": ["my-model-version-01"],
    "inputs": [{"datatype": "BYTES", "name": "input", "shape": [1, 5]}],    
    "outputs": [{"datatype": "BYTES", "name": "output", "shape": [1]}],
}

In [9]:
!kubectl delete -f model-metadata/init-metadata.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


## 2. Via environmental variable

In [10]:
%%writefile model-metadata/environ-metadata.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.3
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
          - name: MODEL_METADATA
            value: |
              ---
              name: my-model-name
              versions: [ my-model-version-01 ]
              platform: seldon
              inputs:
              - datatype: BYTES
                name: input
                shape: [ 1, 4 ]
              outputs:
              - datatype: BYTES
                name: output
                shape: [ 3 ]
    graph:
      children: []
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/environ-metadata.yaml


In [12]:
%%bash

kubectl apply -f model-metadata/environ-metadata.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model configured
deployment "seldon-model-example-0-my-model" successfully rolled out


In [13]:
!curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model | jq .

{
  "inputs": [
    {
      "datatype": "BYTES",
      "name": "input",
      "shape": [
        1,
        4
      ]
    }
  ],
  "name": "my-model-name",
  "outputs": [
    {
      "datatype": "BYTES",
      "name": "output",
      "shape": [
        3
      ]
    }
  ],
  "platform": "seldon",
  "versions": [
    "my-model-version-01"
  ]
}


In [14]:
import json

meta = !curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model 
meta = json.loads(meta[0])

assert meta == {
    "name": "my-model-name",
    "platform": "seldon",
    "versions": ["my-model-version-01"],
    "inputs": [{"datatype": "BYTES", "name": "input", "shape": [1, 4]}],    
    "outputs": [{"datatype": "BYTES", "name": "output", "shape": [3]}],
}

In [15]:
!kubectl delete -f model-metadata/environ-metadata.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


## 3. Via environmental variable (V1 format)

This illustrates how to use v1 format to define metadata.

This example uses environmental variable but this would also be valid for `init_metadata` approach.

In [18]:
%%writefile model-metadata/environ-metadata-v1.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.3
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
          - name: MODEL_METADATA
            value: |
              ---
              name: my-model-name
              versions: [ my-model-version-01 ]
              platform: seldon
              inputs:
              - messagetype: tensor
                schema:
                  names: [a, b, c, d]
                  shape: [ 2, 2 ]
              outputs:
              - messagetype: tensor
                schema:
                  shape: [ 1 ]
    graph:
      children: []
      endpoint:
        type: REST
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/environ-metadata-v1.yaml


In [20]:
%%bash

kubectl apply -f model-metadata/environ-metadata-v1.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model configured
deployment "seldon-model-example-0-my-model" successfully rolled out


In [21]:
!curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model | jq .

{
  "inputs": [
    {
      "messagetype": "tensor",
      "schema": {
        "names": [
          "a",
          "b",
          "c",
          "d"
        ],
        "shape": [
          2,
          2
        ]
      }
    }
  ],
  "name": "my-model-name",
  "outputs": [
    {
      "messagetype": "tensor",
      "schema": {
        "shape": [
          1
        ]
      }
    }
  ],
  "platform": "seldon",
  "versions": [
    "my-model-version-01"
  ]
}


In [27]:
import json

meta = !curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model 
meta = json.loads(meta[0])

assert meta == {
    "name": "my-model-name",
    "platform": "seldon",
    "versions": ["my-model-version-01"],
    "inputs": [{"messagetype": "tensor", "schema": {"names": ["a", "b", "c", "d"], "shape": [2, 2]}}],    
    "outputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],    
}

In [29]:
!kubectl delete -f model-metadata/environ-metadata.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
